In [349]:
import pandas as pd
import numpy as np
import warnings
import scipy.io.arff as arff
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.preprocessing import MultiLabelBinarizer
warnings.filterwarnings('ignore')

In [350]:
whole_data = pd.DataFrame(arff.loadarff('1year.arff')[0])
whole_data = whole_data.replace(b'1', "1")
whole_data = whole_data.replace(b'0', "0")
bankrupt = whole_data[whole_data['class'] == "1"]
non_bankrupt = whole_data[whole_data['class'] == "0"]
random_b = bankrupt.sample(n=100)
random_nb = non_bankrupt.sample(n=100)

In [351]:
vars =  ['Attr3', 'Attr6', 'Attr7', 'Attr8', 'Attr9','class']
vars_attr = vars[:-1]
data = pd.concat([random_b[vars], random_nb[vars]])
data.dropna(inplace=True)
data["Z"] = 1.2*data[vars[0]] + 1.4*data[vars[1]] + 3.3*data[vars[2]] + 0.6*data[vars[3]] + 0.99*data[vars[4]]
data["class_prediction"] = np.where(data["Z"] < 2.67, "1", "0") 
count = len(data[((data['class'] == "1") & (data['class_prediction'] == '1')) | ((data['class'] == "0") & (data['class_prediction'] == '0'))])
print("Poprawność klasyfikacji: {:.2%}".format(count/len(data)))

Poprawność klasyfikacji: 60.00%


In [352]:
X = data.drop(['class', 'Z', 'class_prediction'], axis=1)
Y = pd.DataFrame(data['class'])
lda = LinearDiscriminantAnalysis()
lda.fit(X, Y)
data['lda_z'] = lda.transform(data[vars_attr])
data['lda_predicted_class'] = lda.predict(data[vars_attr])
coef = np.round(lda.coef_, 2)
intercept = np.round(lda.intercept_, 2)

In [353]:
print('Poprawność klasyfikacji = {:.2%}'.format(np.sum((data['class'] == data['lda_predicted_class'])) / len(data)))
print('Wzór funkcji dyskryminacyjnej:',coef[0][0], '* X1 +', coef[0][1], '* X2 +', coef[0][2], '* X3 +', coef[0][3], '* X4 +', coef[0][4], '* X5 +', intercept[0])

Poprawność klasyfikacji = 68.50%
Wzór funkcji dyskryminacyjnej: -0.3 * X1 + -0.69 * X2 + -1.9 * X3 + 0.01 * X4 + -0.08 * X5 + 0.29


In [354]:
#Model Springate'a
#X1 - kapitał pracujący / aktywa ogółem - X3
#X2 - zysk przed spłatą odsetek i podatkiem / aktywa ogółem - X7
#X3 - wynik brutto / zobowiązania krótkoterm - X12
#X4 - przychody ze sprzedaży / aktywa ogółem - X1
vars =  ['Attr3', 'Attr7', 'Attr12', 'Attr1','class']
vars_attr = vars[:-1]
data = pd.concat([random_b[vars], random_nb[vars]])
data.dropna(inplace=True)

data["Z"] = 1.03*data[vars[0]] + 3.07*data[vars[1]] + 0.66*data[vars[2]] + 0.4*data[vars[3]]
data["class_prediction"] = np.where(data["Z"] <= 0.862, "1", "0")

count = len(data[((data['class'] == "1") & (data['class_prediction'] == '1')) | ((data['class'] == "0") & (data['class_prediction'] == '0'))])
print("Poprawność klasyfikacji: {:.2%}".format(count/len(data)))


Poprawność klasyfikacji: 63.50%


In [355]:
#Model B. Prusaka
#X1 - rentowność operacyjna aktywów - zyski operacyjne/aktywa ogółem - X22
#X2 - relacja kosztów operacyjnych do zobowiązań - koszty operacyjne/zobowiązania ogółem - X34
#X3 - płynność bieżąca - aktywa obrotowe/zobowiązania krótkoterminowe - X4
#X4 - rentowność operacyjna sprzedaży - zyski z działalności operacyjnej/przychody netto ze sprzedaży - X42
vars =  ['Attr22', 'Attr34', 'Attr4', 'Attr42','class']
vars_attr = vars[:-1]
data = pd.concat([random_b[vars], random_nb[vars]])
data.dropna(inplace=True)

data["Z"] = 6.5245*data[vars[0]] + 0.148*data[vars[1]] + 0.4061*data[vars[2]] + 2.1754*data[vars[3]] - 1.5685
data["class_prediction"] = np.where(data["Z"] <= -0.13, "1", np.where(data["Z"] > 0.65, "0", "2"))
data = data[data["class_prediction"] != "2"]

count = len(data[((data['class'] == "1") & (data['class_prediction'] == '1')) | ((data['class'] == "0") & (data['class_prediction'] == '0'))])
print("Poprawność klasyfikacji: {:.2%}".format(count/len(data)))

Poprawność klasyfikacji: 65.61%
